In [ ]:
# Import all necessary libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from google.colab import files
import numpy as np

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
INITIAL_EPOCHS = 20
ACCURACY_DELTA_THRESHOLD = 0.005  # Stop if accuracy improvement is less than 0.5% over patience period
MODEL_FILENAME = 'best_model.h5'  # Explicitly set .h5 extension

# Function to setup Kaggle and download dataset
def setup_and_download_dataset():
    print("Setting up Kaggle and downloading dataset...")
    !pip install kaggle -q
    if not os.path.exists('/root/.kaggle/kaggle.json'):
        files.upload()  # Upload your kaggle.json here
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d kushagratandon12/diabetic-retinopathy-balanced -q
    !unzip -q -o diabetic-retinopathy-balanced.zip -d dataset  # -o flag forces overwrite
    print("Dataset downloaded and extracted successfully!")

# Function to create data generators
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        'dataset/content/Diabetic_Balanced_Data/train',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )

    validation_generator = val_datagen.flow_from_directory(
        'dataset/content/Diabetic_Balanced_Data/val',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, validation_generator

# Function to create a robust CNN model
def create_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Main execution
def main():
    # Setup and download dataset
    setup_and_download_dataset()

    # Create data generators
    train_generator, validation_generator = create_data_generators()
    num_classes = len(train_generator.class_indices)

    # Create model
    model = create_model(num_classes)
    model.summary()

    # Define callbacks
    checkpoint = ModelCheckpoint(
        MODEL_FILENAME,  # Saves as .h5
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=8,
        min_delta=ACCURACY_DELTA_THRESHOLD,
        mode='max',
        verbose=1,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=0.00001,
        verbose=1
    )

    # Training parameters
    current_epochs = INITIAL_EPOCHS
    best_accuracy = 0.0
    iteration = 1

    while True:
        print(f"\nTraining iteration {iteration} with {current_epochs} epochs...")

        # Train the model
        history = model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // BATCH_SIZE,
            epochs=current_epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // BATCH_SIZE,
            callbacks=[checkpoint, early_stopping, reduce_lr],
            verbose=1
        )

        # Get the best validation accuracy from this run
        current_best = max(history.history['val_accuracy'])
        print(f"Current best validation accuracy: {current_best:.4f}")

        # Update best accuracy
        if current_best > best_accuracy:
            best_accuracy = current_best
            print(f"New best accuracy achieved: {best_accuracy:.4f}")

        # Check if early stopping was triggered
        if early_stopping.stopped_epoch > 0:
            print("\nTraining stopped: Validation accuracy improvement has plateaued.")
            print(f"Best validation accuracy achieved: {best_accuracy:.4f}")
            print(f"Downloading the best model as {MODEL_FILENAME}...")
            files.download(MODEL_FILENAME)  # Downloads as .h5
            break

        # Continue with more epochs
        print("Continuing training with increased epochs...")
        current_epochs += 10
        iteration += 1

    # Final results
    print("\nTraining completed!")
    print(f"Final best validation accuracy: {best_accuracy:.4f}")
    print(f"Class indices: {train_generator.class_indices}")

if __name__ == "__main__":
    main()

Setting up Kaggle and downloading dataset...
Dataset URL: https://www.kaggle.com/datasets/kushagratandon12/diabetic-retinopathy-balanced
License(s): CC0-1.0
Dataset downloaded and extracted successfully!
Found 34792 images belonging to 5 classes.
Found 9940 images belonging to 5 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 24, 24, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36864)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,874,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │           1,2

 Total params: 19,399,877 (74.00 MB)

...

1087/1087 ━━━━━━━━━━━━━━━━━━━━ 482s 443ms/step - accuracy: 0.3936 - loss: 1.3308 - val_accuracy: 0.3732 - val_loss: 1.4003 - learning_rate: 0.0010
Epoch 4/20
   1/1087 ━━━━━━━━━━━━━━━━━━━━ 1:26 80ms/step - accuracy: 0.3438 - loss: 1.4012
Epoch 4: val_accuracy did not improve from 0.37319
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.3438 - loss: 1.4012 - val_accuracy: 0.3514 - val_loss: 1.5030 - learning_rate: 0.0010
Epoch 5/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.4356 - loss: 1.2688
Epoch 5: val_accuracy did not improve from 0.37319
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 501s 461ms/step - accuracy: 0.4357 - loss: 1.2688 - val_accuracy: 0.2169 - val_loss: 2.3882 - learning_rate: 0.0010
Epoch 6/20
   1/1087 ━━━━━━━━━━━━━━━━━━━━ 1:27 81ms/step - accuracy: 0.4062 - loss: 1.2002
Epoch 6: val_accuracy did not improve from 0.37319

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 24s 22ms/step - accuracy: 0

1087/1087 ━━━━━━━━━━━━━━━━━━━━ 489s 450ms/step - accuracy: 0.4824 - loss: 1.1838 - val_accuracy: 0.5304 - val_loss: 1.0777 - learning_rate: 2.0000e-04
Epoch 8/20
   1/1087 ━━━━━━━━━━━━━━━━━━━━ 1:27 80ms/step - accuracy: 0.5000 - loss: 1.2494
Epoch 8: val_accuracy did not improve from 0.53044
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 24s 22ms/step - accuracy: 0.5000 - loss: 1.2494 - val_accuracy: 0.5271 - val_loss: 1.0861 - learning_rate: 2.0000e-04
Epoch 9/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step - accuracy: 0.5121 - loss: 1.1295
Epoch 9: val_accuracy did not improve from 0.53044
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 502s 462ms/step - accuracy: 0.5121 - loss: 1.1295 - val_accuracy: 0.4360 - val_loss: 1.2809 - learning_rate: 2.0000e-04
Epoch 10/20
   1/1087 ━━━━━━━━━━━━━━━━━━━━ 1:33 86ms/step - accuracy: 0.4375 - loss: 1.0758
Epoch 10: val_accuracy did not improve from 0.53044

Epoch 10: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/ste

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Training completed!
Final best validation accuracy: 0.5304
Class indices: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}
